# Gen3/Terra Data Utility Functions

## Description

This Notebook provides Python functions to facilitate working with Gen3 data in Terra.

TODO Add a proper description.

This is Notebook is currently a **work in progress** and is not yet intended for general availability/use.  
Some of the more important known limitations are:
* Subjects are only included in the resulting consolidated table if there is an entry corresponding to the subject for all tables in the consolidation/merge specification. There is not yet support for the distinction between required and optional tables.
* The `reference_file` table data is not currently included in the consolidated table. Attempting to do so resulted in much duplication in the resulting table. This may be solvable, yet requires investigtion.
* Provide user-friendly/useful error messages when problems occur.
* (Currently) fails to import "large" (e.g., 8,000+ rows) consolidated tables into Terra
* Needs documentation for end-user use, customization and selected implementation details.
* Remove numeric index from resulting dataframe
* Needs automated tests
* ...

That said, it generally seems be working pretty well and should enable continuing development of example workspaces for GWAS and other analysis. Yet, given its early state, some issues should be expected to be found, and reported when they are.
Continuing evoltion depends on feedback, which is welcomed and encouraged.


## Dependencies and Imports

Ensure that a recent version of firecloud is installed.
The version must be 0.16.23 or later for flexible entity support.

In [ ]:
# ! pip install --upgrade firecloud
# ! pip show firecloud
# ! pip install pysnooper

In [ ]:
import io
import os
from datetime import datetime

from firecloud import fiss
import firecloud.api as fapi
import numpy as np
import pandas as pd
# import pysnooper

# Gen3 Data Model Specific Functions

### Create a consolidated data table containing both genomic and phenotypic data

In [ ]:
GEN3_ENTITY_ID_COLUMN = "subject_submitter_id"

In [ ]:
GEN3_GENO_PHENO_MERGE_SPEC = [
    {
        "join_key": "simple_germline_variation",
        "table_names": ["simple_germline_variation", "germline_variation_index"]
    },
    {
        "join_key": "submitted_aligned_reads",
        "table_names": ["submitted_aligned_reads", "aligned_reads_index"]
    },
    {
        "join_key": "read_group",
        "table_names": ["read_group"]
    },
    {
        "join_key": "aliquot",
        "table_names": ["aliquot"]
    },
    {
        "join_key": "sample",
        "table_names": ["sample"]
    },
    {
        "join_key": "subject",
        "table_names": ["subject", "blood_pressure_test", "demographic", "exposure", "lab_result", "medical_history", "medication"]
    },
    {
        "join_key": "study",
        "table_names": ["study"]
    },
    {
        "join_key": "project",
        "table_names": ["project"]
    },
    {
        "join_key": "program",
        "table_names": ["program"]
    }
]

In [ ]:
def consolidate_gen3_geno_pheno_tables(new_table_name: str):
    consolidate_to_terra_table(GEN3_GENO_PHENO_MERGE_SPEC, new_table_name, GEN3_ENTITY_ID_COLUMN)

### Create a consolidated data table containing only genomic (no phenotypic) data

In [ ]:
GEN3_GENO_MERGE_SPEC = [
    {
        "join_key": "simple_germline_variation",
        "table_names": ["simple_germline_variation", "germline_variation_index"]
    },
    {
        "join_key": "submitted_aligned_reads",
        "table_names": ["submitted_aligned_reads", "aligned_reads_index"]
    },
    {
        "join_key": "read_group",
        "table_names": ["read_group"]
    },
    {
        "join_key": "aliquot",
        "table_names": ["aliquot"]
    },
    {
        "join_key": "sample",
        "table_names": ["sample"]
    },
    {
        "join_key": "subject",
        "table_names": ["subject"]
    },
    {
        "join_key": "study",
        "table_names": ["study"]
    },
    {
        "join_key": "project",
        "table_names": ["project"]
    },
    {
        "join_key": "program",
        "table_names": ["program"]
    }
]

In [ ]:
def consolidate_gen3_geno_tables(new_table_name: str):
    consolidate_to_terra_table(GEN3_GENO_MERGE_SPEC, new_table_name, GEN3_ENTITY_ID_COLUMN)

### Create a consolidated data table containing only phenotypic (no genomic) data

In [ ]:
GEN3_PHENO_MERGE_SPEC = [
    {
        "join_key": "subject",
        "table_names": ["subject", "blood_pressure_test", "demographic", "exposure", "lab_result", "medical_history", "medication"]
    },
    {
        "join_key": "study",
        "table_names": ["study"]
    },
    {
        "join_key": "project",
        "table_names": ["project"]
    },
    {
        "join_key": "program",
        "table_names": ["program"]
    }
]

In [ ]:
def consolidate_gen3_pheno_tables(new_table_name: str):
    consolidate_to_terra_table(GEN3_PHENO_MERGE_SPEC, new_table_name, GEN3_ENTITY_ID_COLUMN)

### Enable users to conveniently create a custom consolidtated table, whithout effecting the standard functions above

In [ ]:
GEN3_USER_CUSTOM_MERGE_SPEC = [
    {
        "join_key": "simple_germline_variation",
        "table_names": ["simple_germline_variation", "germline_variation_index"]
    },
    {
        "join_key": "submitted_aligned_reads",
        "table_names": ["submitted_aligned_reads", "aligned_reads_index"]
    },
    {
        "join_key": "read_group",
        "table_names": ["read_group"]
    },
    {
        "join_key": "aliquot",
        "table_names": ["aliquot"]
    },
    {
        "join_key": "sample",
        "table_names": ["sample"]
    },
    {
        "join_key": "subject",
        "table_names": ["subject", "blood_pressure_test", "demographic", "exposure", "lab_result", "medical_history", "medication"]
    },
    {
        "join_key": "study",
        "table_names": ["study"]
    },
    {
        "join_key": "project",
        "table_names": ["project"]
    },
    {
        "join_key": "program",
        "table_names": ["program"]
    }
]

In [ ]:
def consolidate_gen3_custom_tables(new_table_name: str):
    consolidate_to_terra_table(GEN3_USER_CUSTOM_MERGE_SPEC, new_table_name, GEN3_ENTITY_ID_COLUMN)

## Data and functions used internally (not intended for user modification)

In [ ]:
# This is the list of tables defined in the Gen3 data model, for Notebook-internal use.
# All of the tables used in merge specications must exist in this list,
# yet this list may contain additional tables are not used in the
# merge specifications and do not exist in the current workspace data table.
#
# The following information could(/should) be obtained from Gen3 (dynamically?).
# For now, use an explicit list.

GEN3_TABLE_NAMES={'aligned_reads_index',
 'aliquot',
 'blood_pressure_test',
 'demographic',
 'exposure',
 'germline_variation_index',
 'lab_result',
 'medical_history',
 'medication',
 'program',
 'project',                
 'read_group',
 'reference_file',
 'sample',
 'simple_germline_variation',
 'study',
 'subject',
 'submitted_aligned_reads'}

In [ ]:
def consolidate_to_terra_table(merge_spec: list, entity_name: str, entity_id_column:str)  -> pd.DataFrame:
    consolidated_df = consolidate_to_df(merge_spec)
 
    # Add "entity:{entity_name}_id" as the first column, as required by Terra.
    # TODO Check if there is a better way to do this.
    consolidated_df.insert(0, f"entity:{entity_name}_id", consolidated_df[entity_id_column])
    print("The consolidated data frame size is: {} rows x {} columns".format(consolidated_df.shape[0], consolidated_df.shape[1]))
    columns = consolidated_df.columns.tolist()
    write_df_to_tsv_file(consolidated_df, "consolidated_df")
    consolidated_tsv = consolidated_df.to_csv(sep="\t", index=False)
    fiss_entity_import(BILLING_PROJECT_ID, WORKSPACE, consolidated_tsv, "flexible")
    
    # Outout the resulting data table size TODO - Find a more efficient way to do this
    data_table_df = get_terra_table_to_df(BILLING_PROJECT_ID, WORKSPACE, entity_name)
    print("The consolidated data table ""{}"" size is: {} rows x {} columns".format(entity_name, data_table_df.shape[0], data_table_df.shape[1]))

In [ ]:
def consolidate_to_tsv(merge_spec: list)  -> pd.DataFrame:
    return consolidate_to_df(merge_spec).to_csv(sep="\t")

In [ ]:
def consolidate_to_df(merge_spec: list)  -> pd.DataFrame:
    merged_df = None
    for merge_info in merge_spec:
        join_key = get_eid_column_name(merge_info['join_key'])
        merged_df = consolidate_tables_to_df(join_key, merge_info['table_names'], merged_df)
    return merged_df

In [ ]:
# @pysnooper.snoop()
def consolidate_tables_to_df(common_key: str, table_names: list, initial_df = None) -> pd.DataFrame:
    if initial_df is None:
        assert len(table_names) >= 2, "At least two table names are required." 
        table_name = table_names[0]
        merged_df = get_gen3_terra_table_to_df(BILLING_PROJECT_ID, WORKSPACE, table_name)
        if table_name == "sample":
            merged_df = deduplicate_sample_data(merged_df)
        table_names = table_names[1:]
    else:
        assert len(table_names) >= 1, "At least one table names is required to merge with previous data."
        merged_df = initial_df
    for table_name in table_names:
        current_df = get_gen3_terra_table_to_df(BILLING_PROJECT_ID, WORKSPACE, table_name)
        if table_name == "sample":
            current_df = deduplicate_sample_data(current_df)
        # DEBUG -- Comment out the following two lines before committing
        # write_df_to_tsv_file(merged_df, "merged_df")
        # write_df_to_tsv_file(current_df, "current_df")
        merged_df = merged_df.merge(current_df, on=common_key, how="inner", copy=False, suffixes=(False, False))
        # Deduplicate "*_eid" columns
        merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
    return merged_df

In [ ]:
def get_gen3_terra_table_to_df(project: str, workspace: str, table_name: str, model="flexible") -> pd.DataFrame:
    table_df = get_terra_table_to_df(project, workspace, table_name)
    columns = table_df.columns
    rename_column(table_df, f"entity:{table_name}_id", f"{table_name}_eid") # Column 0
    for column in columns[1:]:
        if column in GEN3_TABLE_NAMES:
            rename_column(table_df, column, f"{column}_eid")
        else:
            rename_column(table_df, column, f"{table_name}_{column}")
    # Deduplicate "*_eid" columns
    table_df = table_df.loc[:,~table_df.columns.duplicated()]
    return table_df


In [ ]:
def get_eid_column_name(entity_type: str):
    return f"{entity_type}_eid"

In [ ]:
def deduplicate_sample_data(df: pd.DataFrame)  -> pd.DataFrame:
    # Some TOPMed projects (COPDGene, MESA, maybe others) are known to have multiple sample
    # entries for the same subject. According to BDC data experts, the duplicates should
    # be equivalent, so just keep the first entry found in each case.
    deduped_df = df.drop_duplicates(subset=["subject_eid"], keep="first", inplace=False)
    original_row_count = df.shape[0]
    deduped_row_count = deduped_df.shape[0]
    if deduped_row_count < original_row_count:
        difference = original_row_count - deduped_row_count
        print("Removed {} duplicate rows from \"sample\" data based on subject. Retained the first entry found.".format(difference))
    return deduped_df

## Common/General Utility Functions

In [ ]:
def get_terra_table_to_df(project: str, workspace: str, table_name: str, model="flexible") -> pd.DataFrame:
    table_df = pd.read_csv(io.StringIO(fapi.get_entities_tsv(project, workspace, table_name, model=model).text), sep='\t')
    return table_df

In [ ]:
def rename_column(df: pd.DataFrame, current_column_name: str, new_column_name: str) -> None:
    df.rename(columns={current_column_name : new_column_name}, inplace=True)

In [ ]:
def write_df_to_tsv_file(df: pd.DataFrame, filename: str) -> None:
    filename += "_" + datetime.now().strftime("%Y%m%d_%H%M%S%f") + ".tsv"
    with open(filename, mode="w") as tsv_file:
        tsv_string = df.to_csv(sep="\t", index=False)
        tsv_file.write(tsv_string)

In [ ]:
def fiss_entity_import(project: str, workspace: str, entity_tsv: str, model: str):
    response = fapi.upload_entities(project, workspace, entity_tsv, model)
    fapi._check_response_code(response, 200)

In [ ]:
def delete_terra_table(project: str, workspace: str, table_name: str):
    # TODO There has to be better way than this to simply delete a table/entity-type.
    table_to_delete_df = get_terra_table_to_df(project, workspace, table_name)
    entity_id_column_name = f"entity:{table_name}_id"
    entity_id_series = table_to_delete_df[entity_id_column_name]
    num_chunks = entity_id_series.size / 100
    for chunk in  np.array_split(entity_id_series, num_chunks):
        response = fapi.delete_entity_type(project, workspace, table_name, chunk)
        fapi._check_response_code(response, 204)

# Built-in Test/Debug/Demo Code

To test with data in a different workspace than the one that contains this Notebook,
specify remote workspace information below. This enables convenient testing of data
for multiple different projects/cohorts using this same Notebook in the current workspace.

In [ ]:
# os.environ['GOOGLE_PROJECT'] = os.environ['WORKSPACE_NAMESPACE'] = "anvil-stage-demo"
# os.environ['WORKSPACE_NAME']="mbaumann terra_data_util test Amish"

Set standard names used in this Notebook for these values.

In [ ]:
# BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
# WORKSPACE = os.environ['WORKSPACE_NAME']

## Specify which example tables to create

In [ ]:
# create_example_consolidated_geno_pheno_table=True
# create_example_consolidated_geno_table=False
# create_example_consolidated_pheno_table=False
# create_example_consolidated_custom_table=False
# delete_created_tables=True

## Code to create each table specified above

In [ ]:
# if create_example_consolidated_geno_pheno_table:
#     example_table_name = "example_consolidated_geno_pheno_table"
#     consolidate_gen3_geno_pheno_tables(example_table_name)
#     if delete_created_tables:
#          delete_terra_table(BILLING_PROJECT_ID, WORKSPACE, example_table_name)

In [ ]:
# if create_example_consolidated_geno_table:
#     example_table_name = "example_consolidated_geno_table"
#     consolidate_gen3_geno_tables(example_table_name)
#     if delete_created_tables:
#          delete_terra_table(BILLING_PROJECT_ID, WORKSPACE, example_table_name)

In [ ]:
# if create_example_consolidated_pheno_table:
#     example_table_name = "example_consolidated_pheno_table"
#     consolidate_gen3_pheno_tables(example_table_name)
#     if delete_created_tables:
#          delete_terra_table(BILLING_PROJECT_ID, WORKSPACE, example_table_name)

In [ ]:
# if create_example_consolidated_custom_table:
#     example_table_name = "example_consolidated_custom_table"
#     consolidate_gen3_custom_tables(example_table_name)
#     if delete_created_tables:
#          delete_terra_table(BILLING_PROJECT_ID, WORKSPACE, example_table_name)

In [ ]:
# if create_example_consolidated_custom_table:
#     example_table_name = "example_consolidated_custom_table"
#     consolidate_gen3_custom_tables(example_table_name)
#     if delete_created_tables:
#          delete_terra_table(BILLING_PROJECT_ID, WORKSPACE, example_table_name)